In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [9]:
import machine
a = machine.ADC(machine.Pin(33))
b = machine.ADC(machine.Pin(26))
print(a, b)

Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
ValueError: invalid Pin for ADC


In [68]:
import os
os.remove("main.py")

In [69]:
from machine import I2C, Pin
i2c = I2C(scl=Pin(5, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=100000)
print(i2c.scan())
#a = i2c.readfrom_mem(0x5a, 0x06, 3)
#print(k)

[105]


In [70]:
import ustruct 
k = i2c.readfrom_mem(0x69, 0x03, 2)
ppm = ustruct.unpack("<h", k)[0]
print(ppm)

1546


In [33]:
# figaro sensor
def readCO2():
    # pins are: 1 VDD [square pin], 2 GND, 3 ALARM, 4 PWM, 5 CAD0, 
    #           6 MSEL (Pulldown for I2C mode, checked only on startup), 7 CAL, 8 BUSY, 9 TX/SDA, 10 RC/SCL, 11 NC
    k = i2c.readfrom_mem(0x69, 0x03, 2)
    ppm = ustruct.unpack("<h", k)[0]
    return ppm


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [4]:
from OLED_driver import fbuff, oledshow

In [7]:
help(fbuff)

object <FrameBuffer> is of type FrameBuffer
  fill -- <function>
  fill_rect -- <function>
  pixel -- <function>
  hline -- <function>
  vline -- <function>
  rect -- <function>
  line -- <function>
  blit -- <function>
  scroll -- <function>
  text -- <function>


In [12]:
for i in range(50):
#    fbuff.fill(0)
    fbuff.text("hi there", i, i)
    foledshow()

Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
NameError: name 'foledshow' isn't defined


In [77]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py


Sent 638 lines (23042 bytes) to mqtt_as.py.


In [61]:
from machine import I2C, Pin
import time
from mqtt_as import MQTTClient, config
import uasyncio as asyncio
pled = Pin(2, Pin.OUT)


In [63]:
pled.value(1)

In [2]:
%sendtofile main.py

from machine import I2C, Pin
import time
from mqtt_as import MQTTClient, config
import uasyncio as asyncio

i2c = I2C(scl=Pin(5, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=100000)
pled = Pin(2, Pin.OUT)

co2ppm = 0
co2new = False
co2Time = 0

async def readco2():
    while True:
        global co2ppm, co2new, co2Time
        try:
            k = i2c.readfrom_mem(0x69, 0x03, 2)
            co2ppm = (k[0]+k[1]*256)
            co2new = True
            print(co2ppm)
        except OSError:
            co2new = False
        if co2new:
            pled.value(1)
            await asyncio.sleep_ms(50)
            pled.value(0)
        co2Time = time.ticks_ms()
        await asyncio.sleep_ms(200)
        

# Include any cross-project settings.
config['server'] = "10.3.141.1"
config['ssid'] = 'mqtt-wifi'  # EDIT if you're using ESP32
config['wifi_pw'] = 'bubblino'
config['mqttchannel'] = "dust"

mqttchannelppm = config['mqttchannel']+'/co2ppm'

async def main(client):
    pled.value(1)
    print("Connecting.")
    await client.connect()
    print("Connected.")
    for i in range(11):
        pled.value(i%2)
        await asyncio.sleep_ms(100)

    while True:
        if not co2new:  # no inconsistency problem on this as async
            await client.publish(mqttchannelppm, '%d'%co2ppm, qos=1)
        await asyncio.sleep_ms(200)


MQTTClient.DEBUG = True  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
aloop.create_task(readco2())
try:
    aloop.run_until_complete(main(client))
except Exception:
    pass
client.close()  # Prevent LmacRxBlk:1 errors



Sent 63 lines (1639 bytes) to main.py.


In [3]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.10 on 2019-01-25; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f7713a40358, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
